# Grover's Algorithm

### Three Components
1. Create superposition of 2<sup>n</sup> basis states: 
    * How? apply (H) gate on each qubit 
    * each qubit in state: $\vert0\rangle^{\otimes n}$ 
    * $\otimes n$ means a tensor product of states of n qubits
2. Apply an Oracle operator
    * Oracle operator? applies coefficient -1 to each marked element
    * Why? Allows us to single out elements that we want to know the probability distribution for
3. Apply a Diffusion operator
    * inverts amplitude of all elements 
    * in relation to average overall amplitude

Then: 
1. Put all the components together
2. Apply Oracle and Diffusion operators $O(\sqrt{N = 2^n}))$ times

What does Grover's algorithm do? 
* determines probability of elements marked by Oracle operator

### AIM
1. Implement Grovers algorithm with a quantum circuit
2. Mark elements $000001$ and $101010$ 
3. Determine the probilities of those elements with high probability

In [1]:
%pip install -U -r resources/requirements.txt

from IPython.display import clear_output
clear_output()

## Phase Oracle Guidelines

1. add phase of -1 to all marked elements
2. create identity matrix $2^n$ 
3. change diagonal elements to -1 for desired elements
4. convert unitary matrix --> operator

In [97]:
# PHASE ORACLE
from qiskit.quantum_info import Operator
from qiskit import QuantumCircuit
import numpy as np

# first arg: (n) gives number of qubits in circuit 
# second arg: (indices_to_mark) indices of elements that will be marked -1

# AIM: mark elements of 2^n x 2^n identity matrix 

# def createMatrix(n, indices_to_mark):
#     matrix = []
#     for x in range (2**n): # each row
#         matrix.append([])
#         for y in range (2**n): # each item in row x 
#             matrix[x].append(0)
#     for index in indices_to_mark: # each item in indices_to_mark
#         for i in range (2**n): # each row
#             if (index >= i*(2**n - 1) and i <= (i+1)*(2**n - 1) and 
#             (index % (2**n - 1)) != 0):
#                 currentRowIndex = index % (2**n)
#                 matrix[i][currentRowIndex] = -1
#             elif (index % (2**n - 1) == 0 and index != 0): 
#                 matrix[i][3] = -1
#             elif (index == 0):
#                 matrix[i][0] = -1
#     return matrix

# print(createMatrix(2, [1, 4]))

def createMatrix(n, indices_to_mark):
    matrix = []
    for x in range (2**n): # each row
        matrix.append([])
        for y in range (2**n): # each item in row x 
            matrix[x].append(0)

    # OPTIMAL SOLUTION
    for index in indices_to_mark: 
        # let index = 11
        row = index // (2**n) # 11 / 4 = 2 --> like floor 
        col = index % (2**n) # 11 % 4 = 3
        matrix[row][col] = -1
    return matrix

    # flattened = []
    # for row in matrix: 
    #     for x in row: 
    #         flattened.append(x)
    # for index in indices_to_mark: 
    #     flattened[index] = -1

    # newMatrix = np.reshape(flattened, (2**n, 2**n))
    # return newMatrix

# print(createMatrix(2, [1, 3, 4]))

# Quantum function
def phase_oracle(n, indices_to_mark, name = "Oracle"): 
    qc = QuantumCircuit(n, name=name)
    # create 2^n x 2^n oracle matrix
    # set elements at indices_to_mark to -1, others to 0
    oracle_matrix = []
    for x in range (2**n): # each row
        oracle_matrix.append([])
        for y in range (2**n): # each item in row x 
            oracle_matrix[x].append(0)

    # OPTIMAL SOLUTION
    for index in indices_to_mark: 
        row = index // (2**n)
        col = index % (2**n)
        oracle_matrix[row][col] = -1
    # return oracle_matrix; 
    qc.unitary(Operator(oracle_matrix), range(n))
    return qc